In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
# import seaborn as sns
# from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import math
from sklearn.metrics import accuracy_score
rng = np.random.default_rng(51)


In [2]:
df = pd.read_csv('letter-recognition.data', header=None)


In [3]:
le = preprocessing.LabelEncoder()
for column_name in df.columns:
    df[column_name] = le.fit_transform(df[column_name])
y = df[0]
X = df.loc[:, df.columns != 0]

/Users/khumachbayramova/anaconda3/envs/linear-models/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/khumachbayramova/anaconda3/envs/linear-models/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/khumachbayramova/anaconda3/envs/linear-models/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/khumachbayramova/anaconda3/envs/linear-models/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [5]:
X_train =  X_train.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
X_test = X_test.to_numpy()

In [26]:
def quartic_kernel(u):
    return (15 / 16) * (1 - u**2)**2 if np.abs(u) <= 1 else 0

In [49]:

class ParzenRosenblattClassifier:
    def __init__(self, kernel='gaussian'):
        self.kernel = kernel
        self.h_values = None
        
        if self.kernel == 'gaussian':
            self.kernel_func = self.gaussian_kernel
        elif self.kernel == 'quartic':
            self.kernel_func = self.quartic_kernel
        elif self.kernel == 'epanechnikov':
            self.kernel_func = self.epanechnikov_kernel

    def fit(self, X, y, h_values=None):
        self.X = X
        self.y = y
        self.h_values = h_values

    def leave_one_out(self, h):
        predictions = []
        for i in range(len(self.X)):
            train_X = np.delete(self.X, i, axis=0)
            train_y = np.delete(self.y, i, axis=0)
            test_point = self.X[i]
            predicted_class = self.predict([test_point], h, train_X, train_y)[0]
            predictions.append(predicted_class)

        return accuracy_score(self.y, predictions)

    def select_h(self, h_candidates):
        best_h = None
        best_accuracy = 0

        for h in h_candidates:
            accuracy = self.leave_one_out(h)
            print(f"Accuracy for h={h}: {accuracy}")
            
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_h = h

        return best_h
    
    def epanechnikov_kernel(self, u):
        return 0.75 * (1 - u**2) if np.abs(u) <= 1 else 0

    def quartic_kernel(self, u):
        return (15 / 16) * (1 - u**2)**2 if np.abs(u) <= 1 else 0
    
    def gaussian_kernel(self, u):
        return np.exp(-0.5 * u**2) / (np.sqrt(2 * np.pi))
    
    def get_kernel_args(self, point, h):
        kernel_args = np.array([])
        for x in self.X:
            distance = np.linalg.norm(x - point)
            kernel_args = np.append(kernel_args, distance/h)
        return kernel_args
    
    def get_integral_under_kernel(self, kernel_args, kernel_func):
        split_kernel_args = np.array_split(kernel_args, len(kernel_args) // 50)
        total_integral = 0.0
        for part in split_kernel_args:
            a = min(part)
            b = max(part)
            integral, _ = spi.quad(self.kernel_func, a, b, points=part)
            total_integral += integral
        return total_integral
    
    

    def predict(self, X_new, h=None, train_X=None, train_y=None):
        predictions = []
        if h is None:
            h = self.select_h(self.h_values)

        for point in X_new:
            scores = np.zeros(len(set(self.y)))
            l = self.X.shape[0]
            kernel_args = self.get_kernel_args(point, h)
            v = self.get_integral_under_kernel(kernel_args, self.quartic_kernel)
            
            for label in set(self.y):
                label_indices = np.where(self.y == label)[0]
                label_data = self.X[label_indices]
                distances = np.linalg.norm(label_data - point, axis=1)
                # kernel_args = np.append(kernel_args, distances/h)
                if self.kernel == 'gaussian':
                    weights = np.exp(-0.5 * (distances / h)**2) / ((2 * np.pi)**(len(point)/2) * h**len(point))
                elif self.kernel == 'quartic':
                    weights = np.array([self.quartic_kernel(u / h) for u in distances])
                elif self.kernel == 'epanechnikov':
                    weights = np.array([self.epanechnikov_kernel(u / h) for u in distances])
                else:
                    pass
                
                scores[label] = sum(weights)/(l*v)

 
            predicted_class = np.argmax(scores, axis=0)
            predictions.append(predicted_class)
        
            

        return np.array(predictions)


In [ ]:
classParzen = ParzenRosenblattClassifier(kernel='gaussian')
classParzen.fit(X_train, y_train, h_values=[2, 5])

predictions = classParzen.predict(X_test)

/var/folders/jr/7cpxm2sj4y74r__lw8nqpvfw0000gn/T/ipykernel_38253/4212973441.py:65: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  integral, _ = spi.quad(self.kernel_func, a, b, points=part)


In [15]:
import scipy.integrate as spi
import numpy as np

def gaussian_kernel(u):
    return 1 / np.sqrt(2 * np.pi) * np.exp(-0.5 * u**2)

def epanechnikov_kernel(u):
    return 0.75 * (1 - u**2) if np.abs(u) <= 1 else 0

def quartic_kernel(u):
        return (15 / 16) * (1 - u**2)**2 if np.abs(u) <= 1 else 0

# Вычисление интеграла от гауссовского ядра от -бесконечности до +бесконечности
result, _ = spi.quad(quartic_kernel, -np.inf, np.inf)
result

1.0

In [24]:
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy gaussian test: {accuracy}')

Accuracy: 0.9386666666666666


In [26]:
classParzen = ParzenRosenblattClassifier(kernel='epanechnikov')
classParzen.fit(X_train, y_train, h_values=[20, 1, 0.1, 0.01])

# Работает дольше
predictions = classParzen.predict(X_test)

Accuracy for h=20: 0.449
Accuracy for h=1: 1.0
Accuracy for h=0.1: 1.0
Accuracy for h=0.01: 1.0


In [46]:
predictions = classParzen.predict(X_test, 7)
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy epanechnikov test: {accuracy}')

Accuracy epanechnikov test: 0.866


In [37]:
classParzen = ParzenRosenblattClassifier(kernel='quartic')
classParzen.fit(X_train, y_train, h_values=[5, 0.1, 0.01])

# Работает дольше
predictions = classParzen.predict(X_test)

Accuracy for h=1: 1.0
Accuracy for h=0.1: 1.0
Accuracy for h=0.01: 1.0


In [44]:
predictions_q = classParzen.predict(X_test, 5)
accuracy = accuracy_score(y_test, predictions_q)
print(f'Accuracy  quartic test: {accuracy}')

Accuracy  quartic test: 0.9376666666666666
